In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
import qiskit
from qiskit import QuantumCircuit,execute
from itertools import combinations
import ipyparallel as ipp
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.providers.aer import AerSimulator
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper,ParityMapper
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy

In [2]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates

# Load eg model h1,h2 tensors

In [3]:
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0+eg_h1_1,eg_h2_0+eg_h2_1

# construct qubit Hamiltonian with parity encoding

In [4]:
#preparing commutator pool
def qubitWeightedOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    qubitH=Z2Symmetries.two_qubit_reduction(qubit_op, qubit_op.num_qubits//2)
    return qubitH

# Construct commutator pool

In [5]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)    
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    return commutator_pool

## Group pool into commuting groups

In [6]:
def grouped_pool(pool):
    grouped_pool=[[pool[0]]]
    for i in range(1,len(pool)):
        flag=0
        for j in range(len(grouped_pool)):
            commutation=[grouped_pool[j][k].commute_with(pool[i]) for k in range(len(grouped_pool[j]))]
            if (numpy.all(commutation)==True):
                grouped_pool[j].append(pool[i])
                flag=1
                break
        if (flag==0):
            grouped_pool.append([pool[i]])
    return grouped_pool           


# check if a pool is MCP

In [42]:
def check_mcp(pool):
    sample_list = numpy.arange(len(pool))
    list_combinations = list()
    for n in range(len(sample_list)//2 + 1):
        list_combinations += list(combinations(sample_list, n))
    list_combinations=list_combinations[1:-1]
    list_of_checks=[]
    flag=True #if flag remains 0 then pool is complete , flip flag to 1 if condition not met 
    for l in range(len(list_combinations)):
        group_of_indices=list_combinations[l]
        op_pool=numpy.array(pool)[numpy.array(group_of_indices)]
        other_group=[i for i in range(len(pool)) if i not in group_of_indices]
        complementary_op_pool=numpy.array(pool)[numpy.array(other_group)]
        all_pairs_commutation=[]
        for i in range(len(op_pool)):
            for j in range(len(complementary_op_pool)):
                all_pairs_commutation.append(op_pool[i].commute_with(complementary_op_pool[j]))
        if numpy.allclose(all_pairs_commutation,[True]*len(all_pairs_commutation)):
            flag=False
            break
        else:
            continue
    return flag

# Construct Minimal complete pool from Hamiltonian commutator pool


In [43]:
#Hamiltonian commutator pool
U=7
h1,h2=egBandHamiltonianPartition(U)
qubitH=qubitWeightedOp(h1,h2)
Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
c_pool=commutatorPool(qubitH,stripZs=True)

In [44]:
#group pool into commuting parts
groupedPool=grouped_pool(c_pool)

In [55]:
#constructing trial MCP from grouped commutator pool
new_pool=[groupedPool[0][0],groupedPool[0][1],groupedPool[1][0],groupedPool[1][1],groupedPool[2][0],groupedPool[2][1],
         groupedPool[3][0],groupedPool[3][1],groupedPool[3][2],groupedPool[4][0]]

In [56]:
[new_pool[i].paulis[0][1].to_label() for i in range(len(new_pool))]

['IIIIXY',
 'IIIIYX',
 'IIIXYI',
 'IIIYXI',
 'IIXIYI',
 'IYIIIX',
 'IIXYXX',
 'YXIIII',
 'YXXIIX',
 'IXYIII']

In [57]:
#check if trial pool constructed from Hamiltonian commutator pool is MCP
isMCP=check_mcp(new_pool)
print("Pool is MCP-",isMCP)

Pool is MCP- True


In [58]:
#a particular pool
labels=["IIIXIY","IIXIYI","IIIXYI","IYXIII","IXYIII","IYIIIX","YXIIII","XYIIII","IIXIIY","IIYIIX"]
pool0=[WeightedPauliOperator([[1j,Pauli(labels[i])]]) for i in range(len(labels))]

In [51]:
isMCP=check_mcp(pool0)
print("Pool is MCP-",isMCP)

Pool is MCP- False


In [60]:
#Yongxin's pool constructed from Hamiltonian Commutator pool
labels=["IIIIYX","IIIIXY","IIXIYI","IIIYXI","IIIXYI","IXYIII","IYIIIX","YXIIII","IIXIIY","IIYIIX"]
pool1=[WeightedPauliOperator([[1j,Pauli(labels[i])]]) for i in range(len(labels))]

In [61]:
#check if Yongxin's pool is MCP
isMCP=check_mcp(pool1)
print("Pool is MCP-",isMCP)

Pool is MCP- True


In [62]:
#MCP as defined in qubit adapt VQE paper
labels=['ZZZZZY','ZZZZYI','ZZZYII','ZZYIII','ZYIIII','YIIIII','ZZZIYI','ZZIYII','ZIYIII','IYIIII']
pool2=[WeightedPauliOperator([[1j,Pauli(labels[i])]]) for i in range(len(labels))]

In [59]:
#a control check that qubit adapt Paper's MCP pool satisfies the criteria
isMCP=check_mcp(pool2)
print("Pool is MCP-",isMCP)

Pool is MCP- True


In [52]:
#check if trial pool randomly constructed is MCP
isMCP=check_mcp(pool0)
print("Pool is MCP-",isMCP)

Pool is MCP- False


In [29]:
with open('data_eg_model_parity.txt','r') as f:
    lines=f.readlines()
    labels=[]
    for line in lines:
        if ((line[0]=='I') or (line[0]=='X') or (line[0]=='Y') or (line[0]=='Z')):
            labels.append(line[:-1])
unique_labels=numpy.unique(labels)

In [30]:
unique_labels

array(['IIIIXY', 'IIIIYX', 'IIIYXI', 'IIXIIY', 'IIXIYI', 'IIXXXY',
       'IIXYXX', 'IIYIIX', 'IXYIII', 'IYIIIX', 'IYXIII', 'XXYIIX',
       'YXIIII', 'YXXIIX'], dtype='<U6')

In [31]:
newlabels=['IIIIXY','IIIIYX','IIIYXI','IIXIIY','IIXIYI','IIYIIX', 'IXYIII', 'IYIIIX', 'IYXIII','YXIIII']
pool_gs_ansatz=[WeightedPauliOperator([[1j,Pauli(label)]]) for label in newlabels]

In [488]:
#check if ops constructing ground state forms a MCP
isMCP=check_mcp(pool_gs_ansatz)
print("Pool is MCP-",isMCP)

10
Pool is MCP- True


# Automate creation of MCP's from HC pool and score them

In [12]:
def MCPscore(pool,ref_ansatz):
    def grad(data):
        op,ref_ansatz=data
        ref_ansatz.push_hopping_operator(op)
        circ1=ref_ansatz.construct_circuit(parameters=[numpy.pi/4.])
        circ2=ref_ansatz.construct_circuit(parameters=[-numpy.pi/4.])
        state1=execute(circ1,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
        state2=execute(circ2,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
        E1=state1.conj()@Hmat@state1
        E2=state2.conj()@Hmat@state2
        gradient=(E1-E2)/2
        ref_ansatz.pop_hopping_operator()
        return gradient
    gradient_arr=list(map(grad,[(op,ref_ansatz) for op in pool]))
    score=len(numpy.where(numpy.abs(gradient_arr)>1e-5)[0])
    return score

In [13]:
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()

In [14]:
MCPscore(new_pool,var_form_base)

6

In [48]:
def createMCP(pool):
    new_pool=[pool[0]]
    for i in range(1,10):
        pool_copy=new_pool.copy()
        labels=[op.paulis[0][1].to_label() for op in pool_copy]
        another_pool=[op for op in pool if op.paulis[0][1].to_label() not in labels]
        check_to_append=[check_mcp(pool_copy+[op]) for op in another_pool]
        req_ops=numpy.array(another_pool)[numpy.array(check_to_append)==True]
        labels=[req_ops[i].paulis[0][1].to_label() for i in range(len(req_ops))]
        countGates=[len(numpy.where(numpy.array(list(labels[i]))!='I')[0]) for i in range(len(labels))]
        Op=req_ops[numpy.argsort(countGates)][0]
        pool_copy.append(Op)
        new_pool=pool_copy
    return new_pool    

In [49]:
c_pool[0].paulis[0][1].to_label()

'IIIIXY'

In [50]:
new_pool=createMCP(c_pool)

In [51]:
check_mcp(new_pool)

True

In [52]:
[new_pool[i].paulis[0][1].to_label() for i in range(len(new_pool))]

['IIIIXY',
 'IIIXYI',
 'IIIIYX',
 'IIXYXX',
 'IIIYXI',
 'IIXIIY',
 'IIXIYI',
 'IIYIIX',
 'IXYIII',
 'IYIIIX']

In [88]:
MCPscore(new_pool,var_form_base)

5